In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Completing the Workflow

- Load and prepare data
- Split train and test sets
- Build the model
 - Addition of Cross Validation
- Tune the Parameters
 - Add `GridSearchCV`
- Reflect on the Model

In [2]:
bikeshare = pd.read_csv('data/bikeshare.csv')
weather = pd.get_dummies(bikeshare.weathersit, prefix='weather')


modeldata = bikeshare[['temp', 'hum']].join(weather[['weather_1', 'weather_2', 'weather_3']])
y = bikeshare.casual

In [3]:
#split train-test set
X_train, X_test, y_train, y_test = train_test_split(modeldata, y)

In [7]:
#instantiate, fit, and evaluate the model
lm = LinearRegression()
lm.fit(X_train, y_train)
predictions = lm.predict(X_train)
mse = mean_squared_error(y_train, predictions)

np.sqrt(mse)

40.78627724056598

### Cross-Validation

Rather than using a single training set, we can iterate through a number of splits of the larger training set itself. One approach is called K-folds, where we take a separate fold each time through.  For example, if we perform a cross-validation with 5 folds, we would split the training data 5 times, each time fitting and evaluating a model.  

In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
scores = cross_val_score(lm, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5)

In [10]:
rmse_scores = np.sqrt(-scores)

In [11]:
rmse_scores

array([40.19954311, 40.73731378, 43.32350538, 41.00108099, 38.6145071 ])

In [12]:
rmse_scores.mean()

40.77519007065239

In [13]:
rmse_scores.std()

1.5196244631240068

In [14]:
kf = cross_validation.KFold(len(modeldata), n_folds = 5, shuffle = True)

In [15]:
mse_values = []
scores = []
n= 0
print( "~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf:
    lm = LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
    mse_values.append(mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
    scores.append(lm.score(modeldata, y))
    n+=1
    print( 'Model', n)
    print( 'MSE:', mse_values[n-1])
    print( 'R2:', scores[n-1])

print( "~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print( 'Mean of MSE for all folds:', np.mean(mse_values))
print( 'Mean of R2 for all folds:', np.mean(scores))

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
MSE: 1754.7045797988867
R2: 0.31188643384642
Model 2
MSE: 1767.3528941413747
R2: 0.31188966283358266
Model 3
MSE: 1465.3852307219188
R2: 0.3117009152440636
Model 4
MSE: 1580.3106085711643
R2: 0.31192485666305736
Model 5
MSE: 1803.0062403519232
R2: 0.3119135551998975
~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of MSE for all folds: 1674.1519107170534
Mean of R2 for all folds: 0.3118630847574042


### Grid Search

Now, suppose we are using our regularized methods here.  We want to be able to also experiment with parameters and determine something like our ideal value for $\alpha$ in the `Ridge()` model.  We can feed a list of values to the `GridSearchCV` and it will run through the possible combinations of these using cross validation.

In [17]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [18]:
param_grid = [
    {'alpha': [alpha for alpha in [1, 5, 10, 40]],
    'fit_intercept': [True, False]
    }
]

In [19]:
ridge = Ridge()
grid = GridSearchCV(ridge, param_grid, cv = 5, scoring = 'neg_mean_squared_error')

In [20]:
grid.fit(modeldata, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [1, 5, 10, 40], 'fit_intercept': [True, False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

### Best Fit and Parameters

Now, we can investigate the model performance.  The `best_params_` will give us the ideal parameters from the search, `best_estimator_` gives the full model information, and `cv_results_` will give us full information including performance for each individual model attempt.

In [21]:
grid.best_params_

{'alpha': 40, 'fit_intercept': False}

In [22]:
grid.best_estimator_

Ridge(alpha=40, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [23]:
results = grid.cv_results_

In [25]:
results = zip(results['mean_test_score'], results['params'])

In [26]:
for mean_score, param in results:
    print(np.sqrt(-mean_score), param)

42.19907472429994 {'alpha': 1, 'fit_intercept': True}
42.206081583932566 {'alpha': 1, 'fit_intercept': False}
42.18885870516342 {'alpha': 5, 'fit_intercept': True}
42.18488903631506 {'alpha': 5, 'fit_intercept': False}
42.177685282479935 {'alpha': 10, 'fit_intercept': True}
42.162559420472945 {'alpha': 10, 'fit_intercept': False}
42.15168131166849 {'alpha': 40, 'fit_intercept': True}
42.10058388779278 {'alpha': 40, 'fit_intercept': False}


### Split, Search, Evaluate

In [27]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(modeldata, y)

In [28]:
#Grid Search
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [1, 5, 10, 40], 'fit_intercept': [True, False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [29]:
#Examine results
results = grid.cv_results_
results = zip(results['mean_test_score'], results['params'])
for mean_score, param in results:
    print(np.sqrt(-mean_score), param)

40.98472272735398 {'alpha': 1, 'fit_intercept': True}
40.993051745843275 {'alpha': 1, 'fit_intercept': False}
40.98670862275685 {'alpha': 5, 'fit_intercept': True}
40.99515030170469 {'alpha': 5, 'fit_intercept': False}
40.99117397282172 {'alpha': 10, 'fit_intercept': True}
41.0014927424939 {'alpha': 10, 'fit_intercept': False}
41.06618466313504 {'alpha': 40, 'fit_intercept': True}
41.100071639455365 {'alpha': 40, 'fit_intercept': False}


In [30]:
grid.best_estimator_

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

### Fit on Test

In [31]:
model = Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [32]:
model.fit(X_test, y_test)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [33]:
model.score(X_test, y_test)

0.3130531123834813

In [34]:
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

In [35]:
rmse = np.sqrt(mse)

In [36]:
rmse

40.65424810846637

### California Housing

The dataset comes from sklearn's dataset library.  We want to perform an end to end modeling project where we predict the median house value in a district.  You should follow a workflow similar to what we've encountered to this point, and may go something like:

- Investigate variable types and distributions
- Transformations?
- Collinearity?
- New Features?
 - Perhaps something like Rooms per Household, Bedrooms per Room, Population per Household
- Missing Values?
 - Examine total bedrooms; drop the districts, delete the attribute, or fill values with sensible number (like median) `.fillna()`
- Encode any Categorical Variables
- Feature Scaling? (`MaxMinScaler` or `StandardScaler`)
- Prepare numerical data
- Split Train and Test Set
- Use Cross Validation and Grid Search to explore models on training set
- Determine best model and evaluate on Test set
- Communicate your results including a visualization of the locations of the houses, colored by their median house value and sized by population (`plt.scatter`)

In [51]:
cali_houses = pd.read_csv('data/cali_housing.csv')

In [52]:
cali_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
Unnamed: 0            20640 non-null int64
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), int64(1), object(1)
memory usage: 1.7+ MB


In [57]:
cali_houses = cali_houses.fillna(cali_houses.median())

In [55]:
cali_houses['bed_something'] = cali_houses['total_rooms'] * cali_houses['total_bedrooms']

In [56]:
cali_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 12 columns):
Unnamed: 0            20640 non-null int64
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20640 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
bed_something         20640 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.9+ MB
